In [1]:
import h5py

In [2]:
import numpy as np

In [3]:
from pyiron_atomistics import Project

In [4]:
pr = Project("test")

In [5]:
pr.remove_jobs(recursive=True, silently=True)

In [6]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [7]:
len(structure)

4

In [8]:
job = pr.create.job.Sphinx("sx")

In [9]:
job.structure = structure

In [10]:
job.calc_minimize()

In [11]:
job.run()

The job sx was saved and received the ID: 1


In [12]:
file_name = job.project_hdf5.file_name
file_name

'/home/jovyan/test/sx.h5'

In [13]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

In [14]:
with h5py.File(file_name, 'r') as f:
    f.visititems(print_attrs)

sx
sx/HDF_VERSION
    TITLE: unicode
sx/NAME
    TITLE: unicode
sx/TYPE
    TITLE: unicode
sx/VERSION
    TITLE: unicode
sx/executable
sx/executable/HDF_VERSION
    TITLE: unicode
sx/executable/NAME
    TITLE: unicode
sx/executable/OBJECT
    TITLE: unicode
sx/executable/TYPE
    TITLE: unicode
sx/executable/executable
sx/executable/executable/HDF_VERSION
    TITLE: unicode
sx/executable/executable/NAME
    TITLE: unicode
sx/executable/executable/OBJECT
    TITLE: unicode
sx/executable/executable/READ_ONLY
    TITLE: int
sx/executable/executable/TYPE
    TITLE: unicode
sx/executable/executable/VERSION
    TITLE: unicode
sx/executable/executable/accepted_return_codes__index_5
    TITLE: json
sx/executable/executable/executable__index_3
    TITLE: None
sx/executable/executable/mpi__index_4
    TITLE: int
sx/executable/executable/name__index_1
    TITLE: unicode
sx/executable/executable/operation_system_nt__index_2
    TITLE: int
sx/executable/executable/version__index_0
    TITLE: unicod